## 5.4 自定义层

### 5.4.1 不带参数的层

下面的这个例子没有任何参数。

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

测试其能否正常工作。

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

下面将其组合到更复杂的模型中。

In [4]:
net = nn.Sequential(
    nn.Linear(8, 128),
    CenteredLayer(),
)

我们可以在向该网络发送随机数据后，检查均值是否为0。 由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。

In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-10, grad_fn=<MeanBackward0>)

### 5.4.2 带参数的层

下面实现了一个自定义的全连接层

In [6]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

下面来实例化并访问参数

In [7]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4793, -0.4359, -1.6118],
        [-0.9174,  0.9523, -0.5931],
        [ 1.8113, -1.3309, -0.5868],
        [-0.4058, -1.0117, -1.1958],
        [-0.9856,  0.7335, -0.4040]], requires_grad=True)

也可以使用自定义层进行前向传播

In [18]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.6704, 0.0000],
        [0.0000, 0.2972, 0.0000]])

也可以使用自定义层构建模型

In [23]:
net = nn.Sequential(
    MyLinear(64, 8),
    MyLinear(8, 1)
)
net(torch.rand(2, 64))

tensor([[0.0000],
        [3.5477]])

### 5.4.3 小结

- 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
- 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
- 层可以有局部参数，这些参数可以通过内置函数创建。